<a href="https://colab.research.google.com/github/rayan-roy/Team-Snowflakes/blob/main/Cyclica_challenge_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Walkthrough

The dataset is a standard dataframe importabale through pandas. 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

In [4]:
import pandas as pd

df_train = pd.read_csv("/content/drive/MyDrive/af2_dataset_training_labeled.csv.gz", index_col=0)
df_train.head()

,annotation_sequence,feat_A,feat_C,feat_D,feat_E,feat_F,feat_G,feat_H,feat_I,feat_K,...,feat_DSSP_10,feat_DSSP_11,feat_DSSP_12,feat_DSSP_13,coord_X,coord_Y,coord_Z,entry,entry_index,y_Ligand
0,M,False,False,False,False,False,False,False,False,False,...,0,0.0,47,-0.0,-26.499001,-4.742,-35.189999,GEMI5_HUMAN,0,False
1,G,False,False,False,False,False,True,False,False,False,...,0,0.0,0,0.0,-25.158001,-1.342,-34.104000,GEMI5_HUMAN,1,False
2,Q,False,False,False,False,False,False,False,False,False,...,1,-0.0,-1,-0.0,-21.926001,-1.641,-32.175999,GEMI5_HUMAN,2,False
3,E,False,False,False,True,False,False,False,False,False,...,706,-0.1,705,-0.0,-22.073999,0.654,-29.171000,GEMI5_HUMAN,3,False
4,P,False,False,False,False,False,False,False,False,False,...,0,0.0,705,-0.2,-19.783001,2.670,-26.858999,GEMI5_HUMAN,4,False


In [5]:
#df_train.describe()
#df_train.isna().sum() # only annotation atomrec has missing values

In [5]:
df_train = df_train.drop(columns =['annotation_atomrec','annotation_sequence','entry', 'entry_index'], axis = 1)

In [7]:
df_train.dtypes

feat_A             bool
feat_C             bool
feat_D             bool
feat_E             bool
feat_F             bool
feat_G             bool
feat_H             bool
feat_I             bool
feat_K             bool
feat_L             bool
feat_M             bool
feat_N             bool
feat_P             bool
feat_Q             bool
feat_R             bool
feat_S             bool
feat_T             bool
feat_V             bool
feat_W             bool
feat_Y             bool
feat_PHI        float64
feat_PSI        float64
feat_TAU        float64
feat_THETA      float64
feat_BBSASA     float64
feat_SCSASA     float64
feat_pLDDT      float64
feat_DSSP_H        bool
feat_DSSP_B        bool
feat_DSSP_E        bool
feat_DSSP_G        bool
feat_DSSP_I        bool
feat_DSSP_T        bool
feat_DSSP_S        bool
feat_DSSP_6       int64
feat_DSSP_7     float64
feat_DSSP_8       int64
feat_DSSP_9     float64
feat_DSSP_10      int64
feat_DSSP_11    float64
feat_DSSP_12      int64
feat_DSSP_13    

In [8]:
# They drop categorical data but maybe useful.  Drop the entry and entry index column. Might try out VertexAI to do the modelling aspect.
# If you read the discord, the data is imbalanced. So PR-auc score will be bad, but he said recall is more important than precision.
# He is expecting 0.7 ROC-AUC score
# Column called feat_SCSASA. That should never have negative value. So either drop large neg values or 0 them out? See how much data we will lose

 Lastly (per the Q on supplementary data), the datasets should have an annotation for the protein's entry name, along with XYZ coordinates and the residue's sequence position.  So, properties of the immediate neighbors (in sequence space or 3d space)  may be helpful for inference if you want to consider more complex models

In [9]:
df_train.y_Ligand.value_counts()/df_train.shape[0]
# We see that 96% is false and 3.4% is True (i.e binding). Way too imbalanced dataset

False    0.965295
True     0.034705
Name: y_Ligand, dtype: float64

In [6]:
# It doesn't make sense for the SCSASA values to be negative. Might try to make them 0
df_train[df_train.feat_SCSASA < 0] = 0
# We can see a lot of the values as negative

In [7]:
df_train = df_train.applymap(lambda x: 1 if x == True else x)
df_train = df_train.applymap(lambda x: 0 if x == False else x)

In [15]:
df_train.dtypes

feat_A            int64
feat_C            int64
feat_D            int64
feat_E            int64
feat_F            int64
feat_G            int64
feat_H            int64
feat_I            int64
feat_K            int64
feat_L            int64
feat_M            int64
feat_N            int64
feat_P            int64
feat_Q            int64
feat_R            int64
feat_S            int64
feat_T            int64
feat_V            int64
feat_W            int64
feat_Y            int64
feat_PHI        float64
feat_PSI        float64
feat_TAU        float64
feat_THETA      float64
feat_BBSASA     float64
feat_SCSASA     float64
feat_pLDDT      float64
feat_DSSP_H       int64
feat_DSSP_B       int64
feat_DSSP_E       int64
feat_DSSP_G       int64
feat_DSSP_I       int64
feat_DSSP_T       int64
feat_DSSP_S       int64
feat_DSSP_6       int64
feat_DSSP_7     float64
feat_DSSP_8       int64
feat_DSSP_9     float64
feat_DSSP_10      int64
feat_DSSP_11    float64
feat_DSSP_12      int64
feat_DSSP_13    

Maybe we could add that first, you have to split into training and test set using stratify. Then second, to correct imbalance you eventually need to run oversampling or undersampling on the training set. Many Sklearn classifier has a parameter called class-weight which you can set to balanced. Finally you could also take a more appropriate metric than accuracy for imbalanced dataset. Try, F1 or area under ROC

 New Section

All columns with the `feat_*` prefix are boolean, integer, or float features that describe the residue itself.  These can be used for training a model.  Domain knowledge of these values should not be necessary to participate in the challenge, but we've provided brief descriptions below for anyone who may be interested:

* `feat_[letter]` are one-hot encoded boolean values for each of the 20 possible amino acids.
* `feat_PHI`, `feat_PSI`, `feat_TAU`, `feat_THETA` describe various protein chain bonding angles, computed with [Biopython](https://biopython.org/docs/1.75/api/Bio.PDB.Polypeptide.html).
* `feat_BBSASA`, `feat_SCSASA` describe the solvent accessible surface area, calculated using [FreeSASA](https://freesasa.github.io/).
* `feat_pLDDT` is an AlphaFold2 residue-level prediction confidence value.
* `feat_DSSP_[letter]` are secondary structure assignments by [DSSP].(https://en.wikipedia.org/wiki/DSSP_(algorithm))
* `feat_DSSP_[number]` are other backbone structural features describing backbone hydrogen. bonding networks, also assigned by [DSSP](https://en.wikipedia.org/wiki/DSSP_(algorithm)).

Column `y_Ligand` indicates if the residue (row) belongs to a known binding site or not.  This column is the classification objective for our challenge. 

The remaining columns describe other elements of the protein structure for reference or troubleshooting purposes.  Participants may use this information to to engineer new features/representations in their models if they so choose. These include:
* `annotation_sequence` and `annotation_atomrec`: Residue amino acid in character format.
* `entry`: Protein name, can be looked up on Uniprot for more information about the protein.  Each unique entry is one unique protein structure in this dataset.
* `coord_X`, `coord_Y`, `coord_Z`: XYZ coordinates of the residue in the respective protein structure.  For example, all residues for protein 'QCR1_HUMAN' belong to the same coordinate space, but the coordinate space would shared between two residues (rows) with `entry` values of 'QCR1_HUMAN' and 'PPM1A_HUMAN'.
* `entry_index`: The order of the amino acid within the protein sequence.  As with coordinates, these relationships are only meaningful for rows (residues) that share the same `entry` value.  For example, within QCR1_HUMAN two residues (rows) with `entry_index` 5 and 6 are adjacent (connected) neighbors.

The test dataset has the same format, but is otherwise missing the `y_Ligand` column. 

In [8]:
from sklearn.model_selection import train_test_split

X = df_train.drop(["y_Ligand"], axis=1)
Y = df_train["y_Ligand"].astype('int') #converted to integer
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify = Y)

In [9]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, class_weight='balanced')
rf.fit(X_train,y_train)
y_test_predrf = rf.predict(X_test)

In [10]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, auc, f1_score, roc_curve, roc_auc_score, classification_report, precision_recall_curve

print('Recall on test set: {:.2f}'.format(recall_score(y_test, y_test_predrf))) 
print('Precision on test set: {:.2f}'.format(precision_score(y_test, y_test_predrf))) 
print('ROC/AUC of on test set: {:.2f}'.format(roc_auc_score(y_test, y_test_predrf)))
print('F1 score of on test set: {:.2f}'.format(f1_score(y_test, y_test_predrf, average='weighted'))) # to account for disbalance
precision, recall, _ = precision_recall_curve(y_test, y_test_predrf)
auc_pr = auc(recall, precision)
print(f"PR-AUC {auc_pr}")

Recall on test set: 0.12
Precision on test set: 0.90
ROC/AUC of on test set: 0.56
F1 score of on test set: 0.96
PR-AUC 0.5245604075560165


In [11]:
from imblearn.ensemble import BalancedRandomForestClassifier
BRFC = BalancedRandomForestClassifier(n_estimators=100, class_weight='balanced')
BRFC.fit(X_train,y_train)
y_test_predbrf = BRFC.predict(X_test)



Recall on test set: 0.74
Precision on test set: 0.11
ROC/AUC of on test set: 0.76
F1 score of on test set: 0.85
PR-AUC 0.4302398344085383


In [12]:
from sklearn import metrics
print('Recall on test set: {:.2f}'.format(recall_score(y_test, y_test_predbrf))) 
print('Precision on test set: {:.2f}'.format(precision_score(y_test, y_test_predbrf))) 
print('ROC/AUC of on test set: {:.2f}'.format(roc_auc_score(y_test, y_test_predbrf)))
print('F1 score of on test set: {:.2f}'.format(f1_score(y_test, y_test_predbrf, average='weighted'))) # to account for disbalance
precision, recall, _ = precision_recall_curve(y_test, y_test_predbrf)
auc_pr = auc(recall, precision)
print(f"PR-AUC {auc_pr}")

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_test_predbrf, pos_label=1)
auc_roc = metrics.auc(fpr, tpr)
print("Recall:",metrics.recall_score(y_test, y_test_predbrf,average='binary'))

precision, recall, _ = metrics.precision_recall_curve(y_test, y_test_predbrf)
auc_pr = metrics.auc(recall, precision)

print(f"ROC-AUC for BalancedRF: {auc_roc} \n PR-AUC {auc_pr}")

Recall on test set: 0.74
Precision on test set: 0.11
ROC/AUC of on test set: 0.76
F1 score of on test set: 0.85
PR-AUC 0.4302398344085383
Recall: 0.7431693989071039
ROC-AUC for BalancedRF: 0.7629480644015474 
 PR-AUC 0.4302398344085383


In [19]:
# XgBoost
import xgboost 
xgb = xgboost.XGBClassifier()
xgb.fit(X_train, y_train)
y_test_predxg = xgb.predict(X_test)

print('Recall on test set: {:.2f}'.format(recall_score(y_test, y_test_predxg))) 
print('Precision on test set: {:.2f}'.format(precision_score(y_test, y_test_predxg))) 
print('ROC/AUC of on test set: {:.2f}'.format(roc_auc_score(y_test, y_test_predxg)))
print('F1 score of on test set: {:.2f}'.format(f1_score(y_test, y_test_predxg, average='weighted'))) # to account for disbalance
precision, recall, _ = precision_recall_curve(y_test, y_test_predxg)
auc_pr = auc(recall, precision)
print(f"PR-AUC {auc_pr}")

Recall on test set: 0.00
Precision on test set: 0.92
ROC/AUC of on test set: 0.50
F1 score of on test set: 0.95
PR-AUC 0.4798465320174679


In [20]:
import lightgbm
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(objective='binary')
lgbm.fit(X_train,y_train)
y_test_predlgbm = lgbm.predict(X_test)

print('Recall on test set: {:.2f}'.format(recall_score(y_test, y_test_predlgbm))) 
print('Precision on test set: {:.2f}'.format(precision_score(y_test, y_test_predlgbm))) 
print('ROC/AUC of on test set: {:.2f}'.format(roc_auc_score(y_test, y_test_predlgbm)))
print('F1 score of on test set: {:.2f}'.format(f1_score(y_test, y_test_predlgbm, average='weighted'))) # to account for disbalance
precision, recall, _ = precision_recall_curve(y_test, y_test_predlgbm)
auc_pr = auc(recall, precision)
print(f"PR-AUC {auc_pr}")

Recall on test set: 0.10
Precision on test set: 0.69
ROC/AUC of on test set: 0.55
F1 score of on test set: 0.96
PR-AUC 0.41512560031972234


In [21]:
# Cross validation
from statistics import mean
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scoring = ('f1', 'recall', 'precision', 'roc_auc', 'balanced_accuracy')
#Evaluate BRFC model
scores = cross_validate(BRFC, X, Y, scoring=scoring, cv=cv)
#Get average evaluation metrics
print('Mean f1: %.3f' % mean(scores['test_f1']))
print('Mean recall: %.3f' % mean(scores['test_recall']))
print('Mean precision: %.3f' % mean(scores['test_precision']))

Mean f1: 0.194
Mean recall: 0.760
Mean precision: 0.111


In [31]:
print('Mean ROC-AUC: %.3f' % mean(scores['test_roc_auc']))
print('Mean balanced_accuracy: %.3f' % mean(scores['test_balanced_accuracy']))

Mean ROC-AUC: 0.864
Mean balanced_accuracy: 0.772


In [34]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth' : [10,20,30],
    'criterion' :['gini', 'entropy'],
    'max_features': [15, 20, 30],
    'min_samples_leaf': [1, 2, 3]
}
CV_rfc = GridSearchCV(estimator=BRFC, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
CV_rfc.best_params_

KeyboardInterrupt: ignored

In [13]:
df_true = df_train[df_train["y_Ligand"] == 1]
df_false = df_train[df_train["y_Ligand"] == 0]
df_fsamp = df_false.sample(n=20000, random_state=37)
df_balanced = pd.concat([df_true, df_fsamp])

In [24]:
df_bal_X = df_balanced.drop(["y_Ligand"], axis=1)
df_bal_Y = df_balanced["y_Ligand"]

X_train_sam, X_test_sam, y_train_sam, y_test_sam = train_test_split(df_bal_X, df_bal_Y, test_size=0.3, random_state=42, stratify = df_bal_Y)

In [22]:
BRFC_sam = BalancedRandomForestClassifier(n_estimators=100, class_weight='balanced')
BRFC_sam.fit(X_train_sam,y_train_sam)
y_test_predbrf_sam = BRFC_sam.predict(X_test_sam)

In [23]:
print('Recall on test set: {:.2f}'.format(recall_score(y_test_sam, y_test_predbrf_sam))) 
print('Precision on test set: {:.2f}'.format(precision_score(y_test_sam, y_test_predbrf_sam))) 
print('ROC/AUC of on test set: {:.2f}'.format(roc_auc_score(y_test_sam, y_test_predbrf_sam)))
print('F1 score of on test set: {:.2f}'.format(f1_score(y_test_sam, y_test_predbrf_sam, average='weighted'))) # to account for disbalance
precision, recall, _ = precision_recall_curve(y_test_sam, y_test_predbrf_sam)
auc_pr = auc(recall, precision)
print(f"PR-AUC {auc_pr}")

fpr, tpr, thresholds = metrics.roc_curve(y_test_sam, y_test_predbrf_sam, pos_label=1)
auc_roc = metrics.auc(fpr, tpr)
print("Recall:",metrics.recall_score(y_test_sam, y_test_predbrf_sam,average='binary'))

precision, recall, _ = metrics.precision_recall_curve(y_test_sam, y_test_predbrf_sam)
auc_pr = metrics.auc(recall, precision)

print(f"ROC-AUC for BalancedRF: {auc_roc} \n PR-AUC {auc_pr}")

Recall on test set: 0.75
Precision on test set: 0.11
ROC/AUC of on test set: 0.76
F1 score of on test set: 0.85
PR-AUC 0.4317571914457875
Recall: 0.745023419203747
ROC-AUC for BalancedRF: 0.7649902032176539 
 PR-AUC 0.4317571914457875


## Submission Instructions

- Run inference on the test set and save the inference results as a csv file, the file should look like this
```
id,Predicted
0,True
1,False
2,True
3,False
....
```
- Submit the csv on Kaggle
- Automatic evaluation will be done with ROC-AUC
- Top submissions will be further evaluated by the mean of ROC-AUC and PR-AUC

In [ ]:
df_test = pd.read_csv("af2_dataset_testset_unlabeled.csv.gz", index_col=0)
df_test

In [ ]:
y_test_submission = xgb.predict(df_test)

In [ ]:
s = pd.Series(y_test_submission).astype(bool)
s.name = "Predicted"
s.to_csv("submission.csv")